### European up-and-in barrier call option for Wesfarmers Limited (WES)
**Information for purchase:**  
Strike Price = $80.00  
In-Barrier Level = $100.00  
Expires = 15th September 2027  
#### Step 1: Import Packages  


In [9]:
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install scipy
%pip install yfinance
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.lines import Line2D
import yfinance as yf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Step 2: Monte Carlo Simulation

In [10]:
def up_and_in_barrier_call(S0, K, T, r, sigma, B, M=10000, N=252, random_seed=None):
    # Where:
    # S0: initial stock price
    # K: strike price
    # T: time to maturity in years
    # r: risk-free interest rate
    # sigma: volatility
    # B: barrier level
    # M: number of paths
    # N: number of time steps
    # Seed for random number generator
    if random_seed is not None:
        np.random.seed(random_seed)

# Time step
dt = T / N

# Discount factor for present value
discount_factor = np.exp(-r * T)

# Simulate M paths of GBM
S = np.zeros((N + 1, M))
    S[0] = S0
    for t in range(1, N + 1):
        Z = np.random.normal(0, 1, M)  # Standard normal random variables
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

    # Check if the barrier is breached
    breached = np.any(S > B, axis=0)

    # Calculate payoffs for paths that breach the barrier
    payoffs = np.where(breached, np.maximum(S[-1] - K, 0), 0)

    # Calculate the option price
    option_price = discount_factor * np.mean(payoffs)
    return option_price

IndentationError: unexpected indent (599934190.py, line 23)

### Step 3: Determine Initial Stock Price

In [ ]:
# Define ticker and date
ticker = yf.Ticker("WES.AX")
data = ticker.history(start="2025-05-16", end="2025-05-17")

# Actual values
S0 = data["Close"].iloc[0]
K = 80
T = 2.335  #dont hardcode this
r = 0.05 #?
sigma = 0.2 #?
B = 100
M = 10000
#fix below
barrier_call_price = up_and_in_barrier_call(S0, K, T, r, sigma, B, M, random_seed=42)
print(f"Up-and-In Barrier Call Option Price: ${barrier_call_price:.2f}")

NameError: name 'up_and_in_barrier_call' is not defined